# Business Problem:
### RetailWorld, a well-established retail company with a wide range of products, has been serving customers across various countries. The company prioritizes customer satisfaction and aims to provide a seamless shopping experience. However, RetailWorld is currently facing challenges due to factors such as increased competition, shifting consumer preferences, and rising operational costs. To tackle this challenge, RetailWorld is looking to conduct an analysis of its retail transactions dataset to find ways to increase sales, optimize marketing strategies, and improve overall customer satisfaction.

### 1. Exploratory Data Analysis (EDA): Identify sales trends, popular products, and customer behaviors.
### 2. Predictive Market Basket Analysis: Improve product recommendations and promotions by understanding purchasing patterns.
### 3. Recency, Frequency, Monetary (RFM) Analysis: Segment customers to tailor marketing strategies and increase customer retention.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import random
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/retail1/retail.pkl
/kaggle/input/retail-world-dataset/online_retail_II.xlsx
/kaggle/input/retailworlddata/retailworld.pkl


In [7]:
df = pd.read_pickle('/kaggle/input/retail1/retail.pkl')

In [8]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [9]:
df.shape

(1067371, 8)

The dataset has 1067371 records with 8 attributes

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067371 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 73.3+ MB


In [11]:
print(min(df['Price']), max(df['Price']),min(df['InvoiceDate']), max(df['InvoiceDate']))

-53594.36 38970.0 2009-12-01 07:45:00 2011-12-09 12:50:00


In [12]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,1.067371e+06,1067371,1.067371e+06,824364.000000
mean,9.938898e+00,2011-01-02 21:13:55.394028544,4.649388e+00,15324.638504
min,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,12346.000000
25%,1.000000e+00,2010-07-09 09:46:00,1.250000e+00,13975.000000
50%,3.000000e+00,2010-12-07 15:28:00,2.100000e+00,15255.000000
75%,1.000000e+01,2011-07-22 10:23:00,4.150000e+00,16797.000000
max,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,18287.000000
std,1.727058e+02,NaN,1.235531e+02,1697.464450


In [13]:
#Count the number of missing values in each column
missing = df.isnull().sum()

# Calculate the percentage of missing values for each column
missing_percentages = (missing / df.shape[0]) * 100

print(missing,"\n")
print(missing_percentages.round(2))

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64 

Invoice         0.00
StockCode       0.00
Description     0.41
Quantity        0.00
InvoiceDate     0.00
Price           0.00
Customer ID    22.77
Country         0.00
dtype: float64


In [15]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [17]:
df[['Quantity','Price','Customer ID']].describe()

,Quantity,Price,Customer ID
count,824364.000000,824364.000000,824364.000000
mean,12.414574,3.676800,15324.638504
std,188.976099,70.241388,1697.464450
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13975.000000
50%,5.000000,1.950000,15255.000000
75%,12.000000,3.750000,16797.000000
max,80995.000000,38970.000000,18287.000000


1. Quantity has negative values
2. Price is zero for few items

In [18]:
df[df['Quantity']<0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
823911,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
823995,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
824169,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
824170,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


there are 9839 records containing negative quantity

In [19]:
df = df[df['Quantity']>0]

In [21]:
df[df['Description'].str.startswith("")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
824359,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
824360,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
824361,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
824362,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [22]:
df.loc[:, 'Description'] = df['Description'].str.strip()

In [23]:
df.loc[:,'Description'] = df['Description'].str.capitalize()

In [25]:
df = df.drop_duplicates()

In [26]:
df[df['StockCode'].str.contains('C') == True]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
12,489436,48173C,Door mat black flock,10,2009-12-01 09:06:00,5.95,13078.0,United Kingdom
68,489438,85132C,Charlie and lola figures tins,60,2009-12-01 09:24:00,6.40,18102.0,United Kingdom
127,489445,35916C,Pink felt hanging heart w flower,12,2009-12-01 09:57:00,0.85,17519.0,United Kingdom
138,489445,18097C,White tall porcelain t-light holder,6,2009-12-01 09:57:00,2.55,17519.0,United Kingdom
253,489462,90200C,Blue sweetheart bracelet,3,2009-12-01 10:49:00,4.25,17592.0,United Kingdom
...,...,...,...,...,...,...,...,...
823624,581469,51014C,"Feather pen,coal black",12,2011-12-08 19:28:00,0.39,14606.0,United Kingdom
823827,581478,84997C,Childrens cutlery polkadot blue,4,2011-12-09 08:59:00,4.15,17364.0,United Kingdom
824103,581538,85071C,"Charlie+lola""extremely busy"" sign",1,2011-12-09 11:34:00,0.39,14446.0,United Kingdom
824232,581578,84997C,Childrens cutlery polkadot blue,8,2011-12-09 12:16:00,4.15,12713.0,Germany


C in stock code means returned. We will eliminate such records

In [27]:
df = df[df['StockCode'].str.contains('C') == False]

In [28]:
zero_price = df[df["Price"]==0]
zero_price.shape[0]

12

There are 12 records with zero price. It could be because of either a data entry error or must be a free item. Find out which transactions has those.

In [29]:
df[df["Price"]==0].head(12)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
12998,490727,M,Manual,1,2009-12-07 16:38:00,0.0,17231.0,United Kingdom
63878,497819,TEST001,This is a test product.,5,2010-02-12 14:58:00,0.0,14103.0,United Kingdom
63968,497843,TEST001,This is a test product.,5,2010-02-12 15:47:00,0.0,14827.0,United Kingdom
284923,524181,46000M,Polyester filler pad 45x45cm,648,2010-09-27 16:59:00,0.0,17450.0,United Kingdom
472813,543599,84535B,Fairy cakes notebook a6 size,16,2011-02-10 13:08:00,0.0,17560.0,United Kingdom
622161,561916,M,Manual,1,2011-08-01 11:44:00,0.0,15581.0,United Kingdom
679978,568158,PADS,Pads to match all cushions,1,2011-09-25 12:22:00,0.0,16133.0,United Kingdom
683124,568384,M,Manual,1,2011-09-27 09:46:00,0.0,12748.0,United Kingdom
711112,571035,M,Manual,1,2011-10-13 12:50:00,0.0,12446.0,RSA
745526,574252,M,Manual,1,2011-11-03 13:24:00,0.0,12437.0,France


Drop the records which are neither manual nor a test product.

In [31]:
df = df[~df['StockCode'].isin(['46000M', '84535B', 'PADS'])]

In [32]:
df['Price'].describe()

count    77639.000000
mean         5.602978
std         93.023490
min          0.000000
25%          1.250000
50%          2.100000
75%          3.750000
max      10953.500000
Name: Price, dtype: float64

In [37]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,77639.000000,77639,77639.000000,77639.000000
mean,13.027525,2010-10-27 13:25:53.851543552,5.602978,15345.035987
min,1.000000,2009-12-01 07:45:00,0.000000,12346.000000
25%,2.000000,2010-04-19 13:28:00,1.250000,13975.000000
50%,4.000000,2010-10-08 17:28:00,2.100000,15291.000000
75%,12.000000,2011-04-26 15:52:00,3.750000,16894.000000
max,6696.000000,2011-12-09 12:50:00,10953.500000,18287.000000
std,65.938835,NaN,93.023490,1710.373361


In [38]:
df['Year'] = df['InvoiceDate'].dt.year

In [39]:
df['Month'] = df['InvoiceDate'].dt.month

In [40]:
df.rename(columns = {'Price':"Unit_Price"},inplace = True)

In [41]:
df['Total_Price'] = df['Unit_Price']*df['Quantity']

In [42]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Unit_Price', 'Customer ID', 'Country', 'Year', 'Month', 'Total_Price'],
      dtype='object')

Exploratory Data Analysis (EDA): Identify sales trends, popular products, and customer behaviors.¶

In [43]:
#top 10 products that generated most sales
top_10 = df.groupby('Description')['Total_Price'].sum()
top_10.sort_values(ascending = False).head(10)

Description
White hanging heart t-light holder    247048.01
Manual                                151777.67
Jumbo bag red retrospot               134307.44
Postage                               124648.04
Jumbo bag strawberry                   64127.77
Edwardian parasol natural              54332.61
Edwardian parasol black                45885.07
Wooden frame antique white             39909.17
Set/4 white retro storage cubes        36259.65
Red woolly hottie white heart.         31876.47
Name: Total_Price, dtype: float64

In [44]:
#Most sold products by quantity- top 10
most_sold_top_10 = df.groupby('Description')['Quantity'].sum()
most_sold_top_10.sort_values(ascending = False).head(10)

Description
White hanging heart t-light holder    91757
Jumbo bag red retrospot               74224
Jumbo bag strawberry                  35842
Hanging heart zinc t-light holder     25389
Wrap pink fairy cakes                 16586
Single heart zinc t-light holder      16445
Wooden frame antique white            14368
Jumbo bag red white spotty            13879
Large circular mirror mobile          13566
Wrap english rose                     13198
Name: Quantity, dtype: int64

In [45]:
#Most frequently sold Products
most_frequent_top_10 = df.Description.value_counts().sort_values(ascending = False).head(30).reset_index()
print(most_frequent_top_10)


                            Description  count
0    White hanging heart t-light holder   5016
1               Jumbo bag red retrospot   2641
2                               Postage   1803
3            Wooden frame antique white   1790
4                  Jumbo bag strawberry   1525
5     Hanging heart zinc t-light holder   1150
6        Red woolly hottie white heart.    966
7             Edwardian parasol natural    866
8      Single heart zinc t-light holder    828
9           Pink happy birthday bunting    815
10              Edwardian parasol black    799
11  Knitted union flag hot water bottle    775
12       Cream cupid hearts coat hanger    728
13            Scandinavian reds ribbons    699
14      Pink fairy cake childrens apron    689
15                               Manual    688
16               Tea time party bunting    661
17          Blue happy birthday bunting    634
18                  Tea time oven glove    619
19        Traditional christmas ribbons    575
20   Pack 20 

In [46]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Unit_Price', 'Customer ID', 'Country', 'Year', 'Month', 'Total_Price'],
      dtype='object')

In [48]:
#customers that brought in most sales

most_sales_by_customers = df.groupby('Customer ID')['Total_Price'].sum()
most_sales_by_customers.sort_values(ascending = False).head(10)

Customer ID
13694.0    51690.48
14646.0    44364.68
18102.0    42034.17
14911.0    36109.63
15769.0    32358.72
14156.0    31322.78
16029.0    25214.55
16684.0    23880.18
17511.0    20343.02
13798.0    18578.73
Name: Total_Price, dtype: float64

In [49]:
#customers frequently visit
df['Customer ID'].value_counts().sort_values(ascending = False).head(10)

Customer ID
17841.0    1510
14911.0    1005
14606.0     741
15311.0     710
12748.0     700
17850.0     534
14096.0     498
14298.0     486
13694.0     383
14156.0     379
Name: count, dtype: int64

In [50]:
#top sales by country
top_sales_by_country = df.groupby('Country')['Total_Price'].sum()
top_sales_by_country.sort_values(ascending = False).head(10)

Country
United Kingdom    1676926.93
Germany             72131.53
EIRE                68942.53
France              64662.90
Netherlands         49369.80
Spain               25712.41
Norway              21403.45
Switzerland         14182.05
Singapore           13100.82
Portugal            12385.41
Name: Total_Price, dtype: float64

In [51]:
#top sales by year,month,day
top_sales_by_year = df.groupby('Year')['Total_Price'].sum()
top_sales_by_year.sort_values(ascending = False).head(10)

Year
2010    1202667.64
2011     738634.09
2009     160883.48
Name: Total_Price, dtype: float64

In [52]:
#top sales by year,month,day
top_sales_by_month = df.groupby('Month')['Total_Price'].sum()
top_sales_by_month.sort_values(ascending = False).head(10)

Month
12    242082.05
10    210592.16
11    210067.08
3     204102.43
1     194475.65
9     173888.13
5     159708.33
4     148790.26
6     148133.63
7     139537.20
Name: Total_Price, dtype: float64

In [54]:
#total sales in a day
df['Day'] = df['InvoiceDate'].dt.day_name()

In [55]:
df['Day']

1         Tuesday
2         Tuesday
23        Tuesday
28        Tuesday
29        Tuesday
           ...   
824231     Friday
824295     Friday
824301     Friday
824310     Friday
824363     Friday
Name: Day, Length: 77639, dtype: object

In [56]:
#top sales by year,month,day
top_sales_by_day = df.groupby('Day')['Total_Price'].sum()
top_sales_by_day.sort_values(ascending = False).head(10)

Day
Thursday     429459.43
Tuesday      409421.91
Monday       372878.31
Wednesday    354175.70
Friday       330817.02
Sunday       203006.06
Saturday       2426.78
Name: Total_Price, dtype: float64

In [57]:
trans_per_year = df.groupby('Year')['Invoice'].count()
trans_per_year.sort_values(ascending = False).head(10)

Year
2010    44803
2011    27431
2009     5405
Name: Invoice, dtype: int64

In [58]:
trans_per_month = df.groupby('Month')['Invoice'].count()
trans_per_month.sort_values(ascending = False).head(10)

Month
11    9414
12    9032
10    7622
3     7432
9     6069
6     5900
1     5658
5     5575
4     5403
2     5379
Name: Invoice, dtype: int64

In [59]:
trans_per_day = df.groupby('Day')['Invoice'].count()
trans_per_day.sort_values(ascending = False).head(10)

Day
Thursday     15092
Wednesday    13351
Sunday       13288
Tuesday      13134
Monday       12497
Friday       10192
Saturday        85
Name: Invoice, dtype: int64

In [60]:
#peak hours
df['Hour'] = df['InvoiceDate'].apply(lambda x: x.time())

In [61]:
df['Hour']

1         07:45:00
2         07:45:00
23        09:06:00
28        09:06:00
29        09:06:00
            ...   
824231    12:16:00
824295    12:20:00
824301    12:20:00
824310    12:20:00
824363    12:50:00
Name: Hour, Length: 77639, dtype: object

In [62]:
trans_per_hour = df.groupby('Hour')['Invoice'].count()
trans_per_hour.sort_values(ascending = False)

Hour
13:28:00    338
12:21:00    324
12:42:00    309
13:31:00    306
12:06:00    302
           ... 
07:17:00      1
19:09:00      1
07:12:00      1
20:27:00      1
07:42:00      1
Name: Invoice, Length: 762, dtype: int64

In [63]:
fig = px.histogram(x=df['Hour'], nbins=24, labels={'x': 'Hour of Transaction', 'y': 'Frequency'},
                   title='Histogram of Transaction Hours', opacity=0.7)

# Update x-axis ticks to show each hour
fig.update_xaxes(tickmode='array', tickvals=list(range(1, 25)))

# Show the plot
fig.show()

Peak hour is 12 to 1 pm

In [64]:
df.groupby('Country')['Customer ID'].sum().sort_values(ascending = False)

Country
United Kingdom          1.090061e+09
Germany                 2.230649e+07
EIRE                    2.126248e+07
France                  1.556456e+07
Spain                   5.292722e+06
Netherlands             5.273931e+06
Belgium                 4.380242e+06
Switzerland             3.827394e+06
Portugal                2.571885e+06
Finland                 2.297874e+06
Italy                   1.685873e+06
Sweden                  1.468468e+06
Channel Islands         1.427797e+06
Norway                  1.318274e+06
Cyprus                  1.260069e+06
Australia               1.257180e+06
Austria                 1.193912e+06
Greece                  1.070250e+06
Denmark                 9.900660e+05
Unspecified             9.364310e+05
Poland                  7.779740e+05
Malta                   6.900760e+05
United Arab Emirates    5.198660e+05
USA                     4.942370e+05
Canada                  4.935500e+05
Singapore               3.823200e+05
Japan                   3.5776

Since Uk has the most entries, analyze its records

In [65]:
uk = df[df['Country']=='United Kingdom']

In [66]:
uk.shape

(69972, 13)

In [67]:
#sales by year and month
sales = uk.groupby(['Year','Month'])['Total_Price'].sum().sort_values(ascending = False)
sales

Year  Month
2009  12       145849.46
2010  3        113523.45
      1        100085.41
      11        95737.81
      10        93359.66
      9         89812.12
      6         83480.16
      4         74883.41
      5         72768.45
      2         70646.02
2011  11        69396.08
2010  7         67225.89
      8         63842.93
2011  10        57788.52
      5         55478.10
      9         51783.97
      1         49578.02
2010  12        48052.13
2011  3         47391.23
      7         46178.20
      8         45936.04
      4         43795.84
      6         37500.28
      2         35703.01
      12        17130.74
Name: Total_Price, dtype: float64

October and November have the most sales. Could be due to holidays or special offers in those months

Customer segementation based on RFM Analysis is needed.
RFM stands for Recency, Frequency and Monetary.

Recency is finding the number of days from customer last purchase.

Frequency is the total number of distinct purchases or visits by a customer.

Monetary value is the total money spent by each customer.


In [68]:
max(df['InvoiceDate'])

Timestamp('2011-12-09 12:50:00')

In [69]:
#Now = Dec 13, 2011
import datetime as dt
Now  = dt.datetime(2011,12,13)

In [70]:
r_data = uk.groupby('Customer ID')['InvoiceDate'].max().reset_index()
r_data.columns = ['Customer ID','LastPurchaseDate']
r_data.head()

,Customer ID,LastPurchaseDate
0,12346.0,2010-03-02 13:08:00
1,12608.0,2010-10-31 10:49:00
2,12745.0,2010-08-10 10:14:00
3,12746.0,2010-06-17 10:41:00
4,12747.0,2011-12-07 14:34:00


In [71]:
r_data['Recency'] = (Now - r_data['LastPurchaseDate']).dt.days

In [72]:
r_data.head()

,Customer ID,LastPurchaseDate,Recency
0,12346.0,2010-03-02 13:08:00,650
1,12608.0,2010-10-31 10:49:00,407
2,12745.0,2010-08-10 10:14:00,489
3,12746.0,2010-06-17 10:41:00,543
4,12747.0,2011-12-07 14:34:00,5


In [73]:
r_data.drop('LastPurchaseDate',axis = 1, inplace = True)

In [74]:
r_data

,Customer ID,Recency
0,12346.0,650
1,12608.0,407
2,12745.0,489
3,12746.0,543
4,12747.0,5
...,...,...
4406,18281.0,580
4407,18283.0,6
4408,18284.0,434
4409,18286.0,726


In [75]:
# Drop the duplicate invoice entires to avoid miscalculation
f_data = uk.copy()
f_data.drop_duplicates(subset= ['Customer ID','Invoice'], keep='first', inplace=True)

In [76]:
f_data

,Invoice,StockCode,Description,Quantity,InvoiceDate,Unit_Price,Customer ID,Country,Year,Month,Total_Price,Day,Hour
1,489434,79323P,Pink cherry lights,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,81.00,Tuesday,07:45:00
23,489436,35004B,Set of 3 black flying ducks,12,2009-12-01 09:06:00,4.65,13078.0,United Kingdom,2009,12,55.80,Tuesday,09:06:00
42,489437,84507B,Stripes design monkey doll,6,2009-12-01 09:08:00,2.55,15362.0,United Kingdom,2009,12,15.30,Tuesday,09:08:00
61,489438,84031A,Charlie+lola red hot water bottle,56,2009-12-01 09:24:00,3.00,18102.0,United Kingdom,2009,12,168.00,Tuesday,09:24:00
94,489441,84029E,Red woolly hottie white heart.,36,2009-12-01 09:44:00,2.95,18087.0,United Kingdom,2009,12,106.20,Tuesday,09:44:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
824050,581514,84032A,Charlie+lola pink hot water bottle,9,2011-12-09 11:20:00,0.79,17754.0,United Kingdom,2011,12,7.11,Friday,11:20:00
824073,581516,84029G,Knitted union flag hot water bottle,8,2011-12-09 11:26:00,4.25,14422.0,United Kingdom,2011,12,34.00,Friday,11:26:00
824093,581538,85123A,Cream hanging heart t-light holder,1,2011-12-09 11:34:00,2.95,14446.0,United Kingdom,2011,12,2.95,Friday,11:34:00
824160,581567,84971S,Small heart flowers hook,48,2011-12-09 11:56:00,0.39,16626.0,United Kingdom,2011,12,18.72,Friday,11:56:00


In [77]:
f_data = f_data.groupby('Customer ID')['Invoice'].count().reset_index()

In [78]:
f_data.columns = ['Customer ID','Frequency']
f_data.tail()

,Customer ID,Frequency
4406,18281.0,1
4407,18283.0,17
4408,18284.0,1
4409,18286.0,1
4410,18287.0,5


In [79]:
f_data

,Customer ID,Frequency
0,12346.0,10
1,12608.0,1
2,12745.0,2
3,12746.0,1
4,12747.0,23
...,...,...
4406,18281.0,1
4407,18283.0,17
4408,18284.0,1
4409,18286.0,1


In [80]:
m_data = uk.groupby('Customer ID')['Total_Price'].sum().reset_index()
m_data.columns = ['Customer ID','Monetary']

In [81]:
m_data.head()

,Customer ID,Monetary
0,12346.0,221.35
1,12608.0,64.90
2,12745.0,61.18
3,12746.0,15.90
4,12747.0,1425.10


In [82]:
m_data

,Customer ID,Monetary
0,12346.0,221.35
1,12608.0,64.90
2,12745.0,61.18
3,12746.0,15.90
4,12747.0,1425.10
...,...,...
4406,18281.0,22.50
4407,18283.0,185.82
4408,18284.0,20.40
4409,18286.0,248.65


In [83]:
rf = pd.merge(r_data, f_data, on='Customer ID', how='inner')

rfm = pd.merge(rf, m_data, on='Customer ID', how='inner')

In [84]:
rfm.head()

,Customer ID,Recency,Frequency,Monetary
0,12346.0,650,10,221.35
1,12608.0,407,1,64.90
2,12745.0,489,2,61.18
3,12746.0,543,1,15.90
4,12747.0,5,23,1425.10


In [85]:
rfm['Recency_Score'] = pd.qcut(rfm['Recency'],q= 5,labels= ['5','4','3','2','1'])#most recent - 4
rfm['Frequency_Score'] = pd.qcut(rfm['Frequency'], q= 7,labels= ['1','2','3','4','5'],duplicates = 'drop')# most freq -4
rfm['Monetary_Score'] = pd.qcut(rfm['Monetary'],q= 5,labels= ['1','2','3','4','5'])#most monetary - 4

In [86]:
rfm['RFM_Score'] = rfm.Recency_Score.astype(str)+ rfm.Frequency_Score.astype(str) + rfm.Monetary_Score.astype(str)
rfm.head()

,Customer ID,Recency,Frequency,Monetary,Recency_Score,Frequency_Score,Monetary_Score,RFM_Score
0,12346.0,650,10,221.35,1,5,4,154
1,12608.0,407,1,64.90,2,1,3,213
2,12745.0,489,2,61.18,1,1,3,113
3,12746.0,543,1,15.90,1,1,1,111
4,12747.0,5,23,1425.10,5,5,5,555


In [87]:
rfm['RFM_Score'] = rfm['RFM_Score'].astype('int')

https://help.peelinsights.com/docs/rfm-analysis#build-an-audience-from-rfm-segments

https://nealanalytics.com/blog/customer-segmentation-using-rfm-analysis/

In [88]:
Champions = [555, 554, 544, 545, 454, 455, 445]
Loyal_customers = [543, 444, 435, 355, 354, 345, 344, 335]
Potential_loyalist = [553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323]
Recent_customers = [512, 511, 422, 421, 412, 411, 311]
Promising = [525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413, 414, 415, 315, 314, 313]
Customers_needing_attention = [535, 534, 443, 434, 343, 334, 325, 324]
About_to_sleep = [331, 321, 312, 221, 213]
At_risk = [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124]
Cant_lose_them = [155, 154, 144, 214, 215, 115, 114, 113]
Hibernating = [332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211]
Lost = [111, 112, 121, 131, 141, 151]

In [89]:
Champions1 = 0
Loyal_customers1 = 0
Potential_loyalist1 = 0
Recent_customers1 =0
Promising1 = 0
Customers_needing_attention1 = 0
About_to_sleep1 = 0
At_risk1 = 0
Cant_lose_them1 = 0
Hibernating1 = 0
Lost1 = 0

In [90]:
for value in list(rfm['RFM_Score']):
    if value in Champions:
        Champions1 += 1
    elif value in Loyal_customers:
        Loyal_customers1 += 1
    elif value in Potential_loyalist:
        Potential_loyalist1 += 1       
    elif value in Recent_customers:
        Recent_customers1 += 1
    elif value in Promising:
        Promising1 += 1
    elif value in Customers_needing_attention:
        Customers_needing_attention1 += 1
    elif value in About_to_sleep:
        About_to_sleep1 += 1
    elif value in At_risk:
        At_risk1 += 1
    elif value in Cant_lose_them:
        Cant_lose_them1 += 1
    elif value in Hibernating:
        Hibernating1 += 1 
    else:
        Lost1 += 1




In [91]:
print("Champions: ",Champions1)
print("Loyal_customers: ",Loyal_customers1)
print("Potential_loyalists: ",Potential_loyalist1)
print("Recent_customers: ",Recent_customers1)
print("Promising: ",Promising1)
print("Customers_needing_attention: ",Customers_needing_attention1)
print("About_to_sleep: ",About_to_sleep1)
print("At_risk: ",At_risk1)
print("Cant_lose_them: ",Cant_lose_them1)
print("Hibernating: ",Hibernating1)
print("Lost: ",Lost1)

Champions:  656
Loyal_customers:  315
Potential_loyalists:  282
Recent_customers:  566
Promising:  402
Customers_needing_attention:  266
About_to_sleep:  256
At_risk:  260
Cant_lose_them:  329
Hibernating:  563
Lost:  516


Market Basket Analysis

In [106]:
sample = uk.sample(n=1000, random_state=42)  # Selecting 1000 random rows as a sample

---------------------------------------------------------------------------------------

In [107]:
unstacked = sample.groupby(['Invoice','Description'])['Quantity'].sum().unstack().reset_index().fillna(0)

In [108]:
unstacked.drop(columns = 'Invoice',inplace = True)

In [109]:
unstacked

Description,3d dog picture playing cards,3d sheet of cat stickers,3d sheet of dog stickers,4 blue dinner candles silver flock,4 burgundy wine dinner candles,4 ivory dinner candles silver flock,4 purple flock dinner candles,5 strand glass necklace amber,75 green petit four cases,Amber chunky bead bracelet w strap,...,Wrap pink fairy cakes,Wrap suki and friends,Wrap sweets light pink small bowl,"Wrap, carousel","Wrap,suki and friends",Yellow dragonfly helicopter,Yellow flowers felt handbag kit,Yellow metal chicken heart,Yellow pink flower design big bowl,Yellow shark helicopter
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
def encode_quantity(quantity):
    if quantity <= 0:
        return 0
    if quantity >= 1:
        return 1
basket = unstacked.map(encode_quantity)

In [123]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Step 3: Frequent Itemset Generation

frequent_itemsets = apriori(basket, min_support=0.001, use_colnames=True)#min_support greater than 0.001 gives empty rules df

# Step 4: Association Rule Mining

rules = association_rules(frequent_itemsets, metric="lift")

# Step 5: Rule Evaluation
print(rules)

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



                                          antecedents  \
0                (4 blue dinner candles silver flock)   
1                     (4 purple flock dinner candles)   
2                     (5 strand glass necklace amber)   
3                 (Strawberry scented set/9 t-lights)   
4                         (75 green petit four cases)   
..                                                ...   
89  (White hanging heart t-light holder, Set/6 tur...   
90  (Tea time party bunting, Set/6 turquoise butte...   
91               (White hanging heart t-light holder)   
92                           (Tea time party bunting)   
93               (Set/6 turquoise butterfly t-lights)   

                                          consequents  antecedent support  \
0                     (4 purple flock dinner candles)            0.002083   
1                (4 blue dinner candles silver flock)            0.002083   
2                 (Strawberry scented set/9 t-lights)            0.001042   
3      

In [126]:
rules.sort_values(["support", "confidence","lift"],axis = 0, ascending = False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(5 strand glass necklace amber),(Strawberry scented set/9 t-lights),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
3,(Strawberry scented set/9 t-lights),(5 strand glass necklace amber),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
22,(Red victorian fabric heart box),(Blue victorian fabric heart box),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
23,(Blue victorian fabric heart box),(Red victorian fabric heart box),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
48,(Pink foxglove artiifcial flower),(Ivory rose lampshade),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
49,(Ivory rose lampshade),(Pink foxglove artiifcial flower),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
56,(Origami rose incense/candle set),(Origami jasmine incense/candle set),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
57,(Origami jasmine incense/candle set),(Origami rose incense/candle set),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
64,"(White tea,coffee,sugar jars)",(Red hearts light chain),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0
65,(Red hearts light chain),"(White tea,coffee,sugar jars)",0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.0


In [128]:
# Filter out duplicate rules where antecedents and consequents are exchanged
filtered_rules = []
processed_pairs = set()

for index, row in rules.iterrows():
    antecedents = frozenset(row['antecedents'])
    consequents = frozenset(row['consequents'])
    pair = (antecedents, consequents)
    
    # Check if the pair of antecedents and consequents has already been processed
    if pair in processed_pairs:
        continue
    
    # Check if the reverse pair exists in the DataFrame
    reverse_pair = (consequents, antecedents)
    if reverse_pair in processed_pairs:
        continue
    
    # Add the pair to the set of processed pairs
    processed_pairs.add(pair)
    
    # Add the rule to the filtered set
    filtered_rules.append(row)

# Convert the filtered list of rules back to a DataFrame
filtered_rules_df = pd.DataFrame(filtered_rules)


In [129]:
filtered_rules_df.sort_values(["support", "confidence","lift"],axis = 0, ascending = False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(5 strand glass necklace amber),(Strawberry scented set/9 t-lights),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.000000
22,(Red victorian fabric heart box),(Blue victorian fabric heart box),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.000000
48,(Pink foxglove artiifcial flower),(Ivory rose lampshade),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.000000
56,(Origami rose incense/candle set),(Origami jasmine incense/candle set),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.000000
64,"(White tea,coffee,sugar jars)",(Red hearts light chain),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.000000
84,"(Lavender scented set/9 t-lights, Cinammon set...",(Pink flowers rabbit easter),0.001042,0.001042,0.001042,1.0,960.0,0.001041,inf,1.000000
24,(Blue yellow flower piggy bank),(Pink felt easter egg basket),0.001042,0.002083,0.001042,1.0,480.0,0.001039,inf,0.998957
44,(Green enamel+glass hair comb),(Set of 3 bird light pink feather),0.001042,0.002083,0.001042,1.0,480.0,0.001039,inf,0.998957
52,(Pink flowers rabbit easter),(Lavender scented set/9 t-lights),0.001042,0.003125,0.001042,1.0,320.0,0.001038,inf,0.997914
82,"(Pink flowers rabbit easter, Cinammon set of 9...",(Lavender scented set/9 t-lights),0.001042,0.003125,0.001042,1.0,320.0,0.001038,inf,0.997914
